Script to output the U100 data for the VWIND constraint for CMIP6.  This includes ensemble mean 100hPa zonal mean zonal wind averaged from 20N to 40N for the future-past difference for the ensemble mean and for 1 mem.

In [16]:
import importlib
import pandas as pd
import xarray as xr
import numpy as np
import xesmf as xe
from numpy import nan
import sys
import warnings
import math
from glob import glob

from ecpaper_utils import readdata_utils as read
from ecpaper_utils import calendar_utils as cal
from ecpaper_utils import averaging_utils as avg

importlib.reload(read)
importlib.reload(cal)
importlib.reload(avg)

warnings.filterwarnings('ignore')



Set paths for CMIP6 models (historical and RCP8.5) and the pressure level to us (in Pa)

In [10]:
histpath="/project/cmip6/historical/Amon/"
rcp85path="/project/cmip6/ssp585/Amon/"
plevuse="10000"
pathout="/project/cas/islas/python/ecpaper2020/DATASORT/VWIND/DATA/"

Information ont he models being used is provided in cmip5csvinfo.csv.  This contains information on the models, number of members and whether there's any special order for the member numbers i.e., if they don't simply go from 1 to N.  Read in this info and set up the dates for each period.

In [7]:
cmip6models=pd.read_csv("../cmip6csvinfo.csv")

ybegp = 1979 ; monbegp = 1 ; yendp = 2014 ; monendp = 12 # dates for past period
ybegf = 2070 ; monbegf = 1 ; yendf = 2099 ; monendf = 12 # dates for Future period

# total number of months (used for checking)
nmonthsp = (yendp-ybegp-1)*12 + (12-monbegp+1) + monendp
nmonthsf = (yendf-ybegf-1)*12 + (12-monbegf+1) + monendf

# set up date names
datebegp=str(ybegp)+"-"+str(monbegp).zfill(2)
dateendp=str(yendp)+"-"+str(monendp).zfill(2)
datebegf=str(ybegf)+"-"+str(monbegf).zfill(2)
dateendf=str(yendf)+"-"+str(monendf).zfill(2)

In [14]:
models=cmip6models["Model"]
nmodels=models.size

# define 1 deg greid
grid_out = xr.Dataset({'lat': (['lat'], np.arange(-90,91,1.0)), 'lon': (['lon'], np.arange(0,360,1.0))})
nlon = grid_out["lon"].size ; nlat=grid_out["lat"].size

# define multi-model arrays
upast_em = xr.DataArray(np.zeros([nmodels,nlat]), coords=[models, grid_out['lat']], dims=['Model', 'lat'], name='upast_em')
upast_1mem = xr.DataArray(np.zeros([nmodels,nlat]), coords=[models, grid_out['lat']], dims=['Model', 'lat'], name='upast_1mem')
ufut_em = xr.DataArray(np.zeros([nmodels,nlat]), coords=[models, grid_out['lat']], dims=['Model', 'lat'], name='ufut_em')
ufut_1mem = xr.DataArray(np.zeros([nmodels,nlat]), coords=[models, grid_out['lat']], dims=['Model', 'lat'], name='ufut_1mem')
# loop over models
for index, modname in models.iteritems():
    
    # ---- the past period ----
    nmems=cmip6models.loc[index, "Nmempast"]
    ftype=cmip6models.loc[index, "ftypep"]
    ptype=cmip6models.loc[index, "ptype"]
    
    for imem in range(1, nmems+1, 1): # loop over members
        if (math.isnan(ptype)):
            memstr="r"+str(imem)+"i1p1f"+str(int(ftype))
        else:
            memstr="r"+str(imem)+"i1p"+str(int(ptype))+"f"+str(int(ftype))
            
                    
        # check for a special order
        changeorder = cmip6models.loc[index, "specialorderpast"]
        if (type(changeorder) == str):
            changeordernp=np.array(changeorder.split(","))
            if (math.isnan(ptype)):
                memstr="r"+str(changeordernp[imem-1])+"i1p1f"+str(int(ftype))
            else:
                memstr="r"+str(changeordernp[imem-1])+"i1p"+str(int(ptype))+"f"+str(int(ftype))
            
        print("Processing past for "+modname+" "+memstr+"...")
        histdir=glob(histpath+"ua/"+modname+"/"+memstr+"/*/")
        histdir=histdir[0]
        print(histdir)
        
        # read in meridional wind, concatenate historical to 2005, rcp85 to 2014
        u = read.read_zonalmean_1lev(histdir+"/*.nc", datebegp, dateendp, plevuse) 
            
        # check again
        if (u.time.size != nmonthsp):
            print("something's wrong, nmonthsp="+str(nmonthsp)+" but u has size "+str(v.time.size))
            sys.exit("u past for "+modname+", "+memstr+" doesn't have the right size")
            
        # calculate DJF mean and interpolate
        udjf = cal.season_mean(u, "ua", season="DJF")
        udjf = udjf.dropna('lat')
        udjfinterp = udjf.interp(lat=np.linspace(-90,90,181), method="cubic")
        
        if (imem == 1):
            upast_1mem[index,:] = udjfinterp
            
        upast_em[index,:] = upast_em[index,:] + udjfinterp[:]/float(nmems)
        
    # ---the future period-------
    
    nmems=cmip6models.loc[index, "Nmemfuture"]
    ftype=cmip6models.loc[index, "ftypef"]
    for imem in range(1,nmems+1,1):
        if (math.isnan(ptype)):
            memstr="r"+str(imem)+"i1p1f"+str(int(ftype))
        else:
            memstr="r"+str(imem)+"i1p"+str(int(ptype))+"f"+str(int(ftype))
        
        changeorder=cmip6models.loc[index,"specialorderfuture"]
        if (type(changeorder) == str):
            changeordernp=np.array(changeorder.split(","))
            if (math.isnan(ptype)):
                memstr="r"+str(changeordernp[imem-1])+"i1p1f"+str(int(ftype))
            else:
                memstr="r"+str(changeordernp[imem-1])+"i1p"+str(int(ptype))+"f"+str(int(ftype))
            
        print("Processing future for "+modname+" "+memstr)
        rcp85dir=glob(rcp85path+"/ua/"+modname+"/"+memstr+"/*/")
        rcp85dir=rcp85dir[0]
        u=read.read_zonalmean_1lev(rcp85dir+"/*.nc", datebegf, dateendf, plevuse)
        
        # check the right number of months are there
        if (u.time.size != nmonthsf):
            print("something's wrong, nmonthf="+str(nmonthsf)+" but u has size "+str(v.time.size))
            sys.exit("u future for "+modname+", "+memstr+" doesn't have the right size")
            
        udjf = cal.season_mean(u, "ua", season="DJF")
        udjf = udjf.dropna('lat')
        udjfinterp = udjf.interp(lat=np.linspace(-90,90,181), method="cubic")
        
        if (imem == 1):
            ufut_1mem[index,:]=udjfinterp
            
        ufut_em[index,:] = ufut_em[index,:] + udjfinterp[:]/float(nmems)

Processing past for ACCESS-CM2 r1i1p1f1...
/project/cmip6/historical/Amon/ua/ACCESS-CM2/r1i1p1f1/gn/
Processing past for ACCESS-CM2 r2i1p1f1...
/project/cmip6/historical/Amon/ua/ACCESS-CM2/r2i1p1f1/gn/
Processing future for ACCESS-CM2 r1i1p1f1
Processing past for ACCESS-ESM1-5 r1i1p1f1...
/project/cmip6/historical/Amon/ua/ACCESS-ESM1-5/r1i1p1f1/gn/
Processing past for ACCESS-ESM1-5 r2i1p1f1...
/project/cmip6/historical/Amon/ua/ACCESS-ESM1-5/r2i1p1f1/gn/
Processing past for ACCESS-ESM1-5 r3i1p1f1...
/project/cmip6/historical/Amon/ua/ACCESS-ESM1-5/r3i1p1f1/gn/
Processing future for ACCESS-ESM1-5 r1i1p1f1
Processing future for ACCESS-ESM1-5 r2i1p1f1
Processing future for ACCESS-ESM1-5 r3i1p1f1
Processing past for AWI-CM-1-1-MR r1i1p1f1...
/project/cmip6/historical/Amon/ua/AWI-CM-1-1-MR/r1i1p1f1/gn/
Processing past for AWI-CM-1-1-MR r2i1p1f1...
/project/cmip6/historical/Amon/ua/AWI-CM-1-1-MR/r2i1p1f1/gn/
Processing past for AWI-CM-1-1-MR r3i1p1f1...
/project/cmip6/historical/Amon/ua/AWI-CM

Calculate indices and output to netcdf

In [17]:
# V_SW index
u100_past_em = avg.cosweightlat(upast_em,20,40)
u100_fut_em = avg.cosweightlat(ufut_em,20,40)
u100 = u100_fut_em - u100_past_em
u100.to_netcdf(pathout+"u100_CMIP5.nc")